In [1]:
%cd drive/My\ Drive/SF-GAN

/content/drive/My Drive/SF-GAN


In [2]:
data_path = "dataset/icdar13/"

In [ ]:
import pathlib
import os
import numpy as np

def read_data(data_path):
    if not os.path.exists(data_path):
        print("%s path not exist!" % data_path)
        return

    data_root = pathlib.Path(data_path)
    imagesRoot = data_root / 'ch4_training_images'
    gtRoot = data_root / 'ch4_training_localization_transcription_gt'
    all_bboxs = []
    all_texts = []
    all_images = []

    for image in imagesRoot.glob('*.jpg'):
        all_images.append(image)
        gt = gtRoot / image.with_name('gt_{}'.format(image.stem)).with_suffix('.txt').name
        with gt.open(mode='r') as f:
            bboxes = []
            texts = []
            for line in f:
                text = line.strip('\ufeff').strip('\xef\xbb\xbf').strip().split(',')
                x1, y1, x2, y2, x3, y3, x4, y4 = list(map(float, text[:8]))
                bbox = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
                transcript = text[8]
                bboxes.append(bbox)
                texts.append(transcript)
            bboxes = np.array(bboxes)
            all_bboxs.append(bboxes)
            all_texts.append(texts)
    return all_images, all_bboxs, all_texts

all_images, all_bboxs, all_texts = read_data(data_path)

ValueError: ignored

In [ ]:
len(all_images)

1000

In [ ]:
import cv2
import matplotlib.pyplot as plt

for i, img in enumerate(all_images):
    image = cv2.imread(str(img))
    height = image.shape[0]
    width = image.shape[1]

    bboxes = all_bboxs[i]
    sz = len(bboxes)
    
    mask = np.zeros(image.shape[:-1], dtype=np.uint8)
    for j, bbox in enumerate(bboxes):
        bbox = np.int32(bbox)
        cv2.fillConvexPoly(mask, np.int32(bbox), (255, ))
    
    background = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)

    for j, bbox in enumerate(bboxes):
        [[x1, y1], [x2, y2], [x3, y3], [x4, y4]] = bbox
        bbox = np.int32(bbox)
        
        x, y, w, h = cv2.boundingRect(bbox) # returns (x,y,w,h) of the rect
        dw = int(w * 0.25)
        dh = int(w * 0.25)

        cropped = image[max(y-dh, 0) : min(y+h+dh, height), max(x-dw, 0): min(x+w+dw, width)]
        cropped_bg = background[max(y-dh, 0) : min(y+h+dh, height), max(x-dw, 0): min(x+w+dw, width)]
        
        # plt.figure()
        # plt.imshow(cropped)
        # plt.figure()
        # plt.imshow(cropped_bg)

        # plt.plot([x1, x2], [y1, y2], color='red', linestyle='-')
        # plt.plot([x2, x3], [y2, y3], color='red', linestyle='-')
        # plt.plot([x3, x4], [y3, y4], color='red', linestyle='-')
        # plt.plot([x4, x1], [y4, y1], color='red', linestyle='-')

        if (h >= 120 or w >= 120) and (all_texts[i][j] != "###"):
            # plt.figure()
            # plt.imshow(cropped)
            # plt.figure()
            # plt.imshow(cropped_bg)

            # plt.figure()
            # plt.title(all_texts[i][j]+" "+str(cropped.shape))
            # plt.imshow(cropped)
            plt.imsave("datasets/background_updated/" + "img_" + str(i+1) + "_" + str(j+1) + ".jpg", cropped_bg)
        # break
    # break


In [ ]:
import torchvision.transforms as transforms
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()

In [ ]:
# from PIL import Image, ImageFont, ImageDraw
# import random
# import glob, os
# import torchvision.transforms as transforms

# class Data:
#     def __init__(self, root):
#         self.fonts = glob.glob(root + 'data/fonts' + '/*.*')
#         self.WORDS = self.load_words(root)
#         assert len(self.fonts) > 0 and len(self.WORDS) > 0
#         ts_list = [
#            transforms.RandomAffine(degrees=20, translate=(0.15, 0.15), scale=(0.7, 1.2), shear=[1, 20, 1, 20], fillcolor='white'), 
#            transforms.RandomPerspective(fill=(255, 255, 255))
#            ]
#         self.transform = transforms.Compose(ts_list)

#     def load_words(self, root):
#         filename = os.path.join(root, 'data/words.txt')
#         WORDS = []
#         with open(filename, 'r') as f:
#             for line in f:
#                 WORDS += line.split()
#         return WORDS

#     def get_random_font(self):
#         font_id = random.randint(0, len(self.fonts) - 1) 
#         return self.fonts[font_id]

#     def get_random_word(self):
#         word_id = random.randint(0, len(self.WORDS) - 1) 
#         return self.WORDS[word_id]

#     def get_random_color(self):
#         r = random.randint(0, 255)
#         g = random.randint(0, 255)
#         b = random.randint(0, 255)
#         color = (r, g, b)
#         inverse_color = (255-r, 255-g, 255-b)
#         return color, inverse_color

#     def get_background_image(self, shape=(256, 256)):
#         start = random.randint(10, 100)
#         h = random.randint(20, 60)
#         w = random.randint(100, 200)
#         xy = [start, start, start+w, start+h]
#         mask = Image.new('RGB', shape, color='white')
#         draw = ImageDraw.Draw(mask)
#         draw.rectangle(xy, fill='yellow')
#         return self.transform(mask)

#     def get_text_mask(self, shape=(256, 256), img_fraction=0.7):
#         font_name = self.get_random_font()
#         word = self.get_random_word()
#         fontsize = 10
#         font = ImageFont.truetype(font = font_name, size=fontsize)

#         breakpoint = img_fraction * shape[0]
#         jumpsize = 10
#         while True:
#             if font.getsize(word)[0] < breakpoint:
#                 fontsize += jumpsize
#             else:
#                 jumpsize = int(jumpsize / 2)
#                 fontsize -= jumpsize
#             font = ImageFont.truetype(font = font_name, size=fontsize)
#             if jumpsize <= 1:
#                 break

#         mask = Image.new('1', shape)
#         draw = ImageDraw.Draw(mask)
#         y = (shape[0] - font.getsize(word)[0])/2
#         x = (shape[1] - font.getsize(word)[1])/2
#         draw.text((y, x), word, font=font, fill='white')
#         return mask    

#     def get_real_image(self, shape=(256, 256)):
#         font_name = self.get_random_font()
#         word = self.get_random_word()
#         fontsize = random.randint(20, 40)
#         font = ImageFont.truetype(font = font_name, size=fontsize)
#         color_bg, color_fg = self.get_random_color()

#         mask = Image.new('RGB', shape, color='white')
#         draw = ImageDraw.Draw(mask)
#         y = (shape[0] - font.getsize(word)[0])/2
#         x = (shape[1] - font.getsize(word)[1])/2
#         draw.rectangle([y-2, x-2, y+font.getsize(word)[0]+1, x+ font.getsize(word)[1]+1], fill='yellow')
#         draw.text((y, x), word, font=font, fill='black')
#         return self.transform(mask)


In [ ]:
# data = Data("/content/drive/My Drive/SF-GAN/")
# for i in range(1000):
#     bg = data.get_background_image()
#     fg = data.get_text_mask()    
#     real = data.get_real_image()
#     bg.save("dataset/background_text/" + str(i+1) + ".jpg")
#     fg.save("dataset/foreground_text/" + str(i+1) + ".jpg")
#     real.save("dataset/real_text/" + str(i+1) + ".jpg")

In [3]:
import pathlib
import os
import numpy as np

def read_data(data_path):
    if not os.path.exists(data_path):
        print("%s path not exist!" % data_path)
        return

    data_root = pathlib.Path(data_path)
    imagesRoot = data_root / 'training_images'
    gtRoot = data_root / 'training_gt'
    maskRoot = data_root / 'training_mask'
    all_bboxs = []
    all_texts = []
    all_images = []
    all_masks = []

    for image in imagesRoot.glob('*.jpg'):
        all_images.append(image)
        gt = gtRoot / image.with_name('gt_{}'.format(image.stem)).with_suffix('.txt').name
        mask = maskRoot / image.with_name('{}_GT'.format(image.stem)).with_suffix('.bmp').name
        all_masks.append(mask)

        with gt.open(mode='r') as f:
            bboxes = []
            texts = []
            for line in f:
                # print(line)
                text = line.split()
                # text = line.strip('\ufeff').strip('\xef\xbb\xbf').strip().split(',')
                # print(text[:4])
                # break
                left, top, right, bottom = list(map(float, text[:4]))
                x1, y1 = left, top
                x2, y2 = right, top
                x3, y3 = right, bottom
                x4, y4 = left, bottom
                bbox = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
                transcript = text[4]
                bboxes.append(bbox)
                texts.append(transcript)
            bboxes = np.array(bboxes)
            all_bboxs.append(bboxes)
            all_texts.append(texts)
    return all_images, all_bboxs, all_texts, all_masks

all_images, all_bboxs, all_texts, all_masks = read_data(data_path)

In [ ]:
len(all_images)

228

In [8]:
import cv2
import matplotlib.pyplot as plt
import random

# r = list(range(len(all_images)))
# random.shuffle(r)
# for i in r:
#     img = all_images[i]

for i, img in enumerate(all_images):
    if i == 121:
        continue
    image = cv2.imread(str(img))
    height = image.shape[0]
    width = image.shape[1]

    bboxes = all_bboxs[i]
    sz = len(bboxes)
    
    # # print(all_masks[i])
    # mask = cv2.imread(str(all_masks[i]))[:,:,-1]
    # mask[mask < 255] = 254
    # mask[mask == 255] = 0
    # mask[mask == 254] = 1
    
    mask = np.zeros(image.shape[:-1], dtype=np.uint8)
    for j, bbox in enumerate(bboxes):
        bbox = np.int32(bbox)
        cv2.fillConvexPoly(mask, np.int32(bbox), (255, ))
    
    background = cv2.inpaint(image, np.uint8(mask), 3, cv2.INPAINT_TELEA)
        
    for j, bbox in enumerate(bboxes):
        bbox = np.int32(bbox)
        x, y, w, h = cv2.boundingRect(bbox)
        masked_image = background[y:y+h, x:x+w]
        blurred = cv2.blur(masked_image, (15,15))
        background[y:y+h, x:x+w] = blurred 


    # blurred = cv2.blur(masked_image, (25,25))
    # background = image.copy()
    # background[blurred > 0] = blurred[blurred > 0] 

    # plt.figure()
    # plt.subplot(1, 2, 1)
    # plt.imshow(image)
    # plt.subplot(1, 2, 2)
    # plt.imshow(background)
    # plt.show()
    # if(i==3):
    #     break
    # continue

    print(i)

    for j, bbox in enumerate(bboxes):
        [[x1, y1], [x2, y2], [x3, y3], [x4, y4]] = bbox
        bbox = np.int32(bbox)
        
        x, y, w, h = cv2.boundingRect(bbox) # returns (x,y,w,h) of the rect
        dw = int(w * 0.5)
        dh = int(w * 0.5)

        # mask = np.zeros(image.shape[:-1], dtype=np.uint8)
        # cv2.fillConvexPoly(mask, np.int32(bbox), (255, ))
        # background = cv2.inpaint(image, np.uint8(mask), 3, cv2.INPAINT_TELEA)        
        # masked_image = background[y:y+h, x:x+w]
        # blurred = cv2.blur(masked_image, (15,15))
        # background = image.copy()
        # background[y:y+h, x:x+w] = blurred 

        cropped = image[max(y-dh, 0) : min(y+h+dh, height), max(x-dw, 0): min(x+w+dw, width)]
        cropped_bg = background[max(y-dh, 0) : min(y+h+dh, height), max(x-dw, 0): min(x+w+dw, width)]

        if (h >= 120 or w >= 120) and (all_texts[i][j] != "###"):
            # plt.figure()
            # plt.subplot(1, 2, 1)
            # plt.imshow(cropped)
            # plt.subplot(1, 2, 1)
            # plt.imshow(cropped_bg)
            try:
                plt.imsave("dataset/base_icdar13/background_updated/" + "img_" + str(i+1) + "_" + str(j+1) + ".jpg", cropped_bg)
                # plt.imsave("dataset/base_icdar13/real/" + "img_" + str(i+1) + "_" + str(j+1) + ".jpg", cropped)
            except:
                print("Exception occured at index ", i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
